### Initialise

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from IPython.display import display_html
from cryptography.fernet import Fernet
import os
from io import StringIO

from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format ='retina'

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

data = "current_data"

folio_key = os.getenv("FOLIO_KEY")
f = Fernet(folio_key)

def decrypt(file_path):
    with open(file_path, "rb") as file:
        file_data = file.read()
    raw = f.decrypt(file_data)
    raw_str = raw.decode('utf-8')
    return pd.read_csv(StringIO(raw_str))

### Account Info

In [ ]:
raw = decrypt(f'{data}/account.csv')
# convert to pandas
columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "Name",
        "AccountType", 
        "CustomerType", 
        "AccountCapabilities",
        "TradingPermissions",
        "DateOpened",
        "DateFunded",
        "Street",
        "Street2",
        "City",
        "State",
        "Country",
        "PostalCode",
        "IBEntity",
        "PrimaryEmail",
    ]
account = raw[columns].copy()
account.head()

### Cash Report

In [ ]:
crtt = decrypt(f'{data}/CRTT.csv')

available_cash = np.round(crtt[crtt['CurrencyPrimary'] == 'BASE_SUMMARY']['NetCashBalanceSLB'].iloc[0], 2)
print(available_cash)
crtt.head()

### Cash Transactions (CTRN)

In [ ]:
ctrn = decrypt(f'{data}/CTRN.csv')
ctrn.head(20)

In [ ]:
# Dataframe condition
# crtt[(crtt["CurrencyPrimary"] == "USD") & (crtt["Type"] == "Deposits/Withdrawals")]

# Selecting multipel columsn from output
# crtt.query('CurrencyPrimary == "USD" and Type == "Deposits/Withdrawals"')[["Amount", "Type"]]

# Selecting via query style
ctrn.query('CurrencyPrimary == "USD" and Type == "Deposits/Withdrawals"').head(40)

bank_transfers = ctrn.query('CurrencyPrimary == "USD" and Type == "Deposits/Withdrawals"')["Amount"].sum()
interest_recieved = ctrn.query('CurrencyPrimary == "USD" and Type == "Broker Interest Received"')["Amount"].sum()

printmd(f"<h3><b><span style='color:#FFCE1B'>Bank Transferes </b> : </span> $ {bank_transfers}</h3")
printmd(f"<h3><b><span style='color:#C04657'>IB Interest Recieved </b> : </span> $ {interest_recieved}</h3")


### Positions

In [ ]:
positions = decrypt(f'{data}/POST.csv')
positions.head(20)

### Trades

In [ ]:
trades = decrypt(f'{data}/TRNT.csv')
trades.head()

### Portfolio

In [ ]:
def calculate(symbol):
    orders = trades[(trades["Symbol"] == symbol) & (trades["LevelOfDetail"] == "EXECUTION")]
    cost_basis = orders["CostBasis"].sum()
    fees = abs(orders["Taxes"].sum()) + abs(orders["IBCommission"].sum())
    pnl = orders[(orders["Buy/Sell"] == "SELL")]["FifoPnlRealized"].sum()

    return np.round(cost_basis, 2), np.round(pnl, 2), np.round(fees, 2)

def get_info(symbol):
    row = trades[(trades["Symbol"] == symbol) & (trades["LevelOfDetail"] == "EXECUTION")].iloc[0]
    return row["Description"], row["ListingExchange"]

def buy_sell_trades(symbol):
   orders = trades[(trades["Symbol"] == symbol) & (trades["LevelOfDetail"] == "EXECUTION")]
   total_sell = abs(orders[(orders["Buy/Sell"] == "SELL")]["Quantity"].sum())
   total_buy = orders[(orders["Buy/Sell"] == "BUY")]["Quantity"].sum()
   current_quantity = orders["Quantity"].sum()
   return np.round(current_quantity, 1), np.round(total_buy, 1), np.round(total_sell, 1)

def total_profits(portfolio):
    total = portfolio['PnL'].sum()

def squash(portfolio, new_symbol, old_symbol):
    # Select rows to merge
    new_asset = portfolio[portfolio['SYMBOL'] == new_symbol].iloc[0]
    old_asset = portfolio[portfolio['SYMBOL'] == old_symbol].iloc[0]

    # Sum numerical values
    squash_asset = new_asset[['CostBasis', 'Quantity', 'Total Buy', 'Total Sell','PnL', 'Fees']] + old_asset[['CostBasis', 'Quantity', 'Total Buy', 'Total Sell','PnL', 'Fees']]

    # Combine non-numerical values (take from new_symbol)
    squash_asset['SYMBOL'] = new_symbol
    squash_asset['Description'] = new_asset['Description']
    squash_asset['Exchange'] = new_asset['Exchange']

    # Drop the row of old and new asset reocrds. Concat the squashed asset
    portfolio = portfolio[~portfolio['SYMBOL'].isin([new_symbol, old_symbol])]
    portfolio = pd.concat([portfolio, pd.DataFrame([squash_asset])])

    return portfolio

def categorised_portfolio(portfolio, category_symbols):
    cat_folio = portfolio[portfolio['SYMBOL'].isin(category_symbols)]
    cat_folio.reset_index(drop=True, inplace=True)
    return cat_folio

def table(folio):
    # table
    html_table = folio.to_html(index=False)
    display_html(html_table, raw=True)

def render(cat_folio, title):
    table(cat_folio)

    # # Graph
    # plt.figure(figsize=(8, 8))
    # plt.pie(cat_folio['CostBasis'], labels=cat_folio['SYMBOL'], autopct='%1.1f%%', startangle=140)
    # plt.title(title)
    # plt.show()

   # Pie chart
    sizes = cat_folio['CostBasis']
    labels = cat_folio['SYMBOL']
    #explsion
    explode = [0.05] * len(labels)
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
    #draw circle
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Equal aspect ratio ensures that pie is drawn as a circle
    plt.axis('equal')  
    plt.tight_layout()
    plt.show()
    

"""
START OF THE MAIN FLOW
"""
symbols = trades[trades["AssetClass"] == 'STK'].Symbol.unique()
growth_symbols = ['VIR', 'TSLA', 'PLL', 'MU', 'IREN', 'CLSK', 'AMD']
emerging_symbols = ['BTDR', 'IRD']
divident_symbols = ['MRK']
data = []

for symbol in symbols:
    cost, pnl, fees = calculate(symbol)
    description, exchange = get_info(symbol)
    current_quantity, total_buy, total_sell = buy_sell_trades(symbol)
    data.append([symbol, description, exchange, cost, current_quantity, total_buy, total_sell, pnl, fees])

portfolio = pd.DataFrame(data, columns=['SYMBOL','Description', 'Exchange', 'CostBasis', 'Quantity', 'Total Buy', 'Total Sell','PnL', 'Fees'])

total_pnl = portfolio['PnL'].sum()
total_fees = portfolio['Fees'].sum()
equities = portfolio['CostBasis'].sum()

# Merge equities that has gone through an aquisition
portfolio = squash(portfolio, "IRD", "OCUP")
portfolio = portfolio.sort_values(by='SYMBOL')

printmd(f"<h1><span style='color:white'>🔘 n03an's Portfolio 🔘</span></h1")
printmd(f"<h3><span style='color:#FFCE1B'>😰 Total Investment (USD) : </span> $ {bank_transfers}</h3")
printmd(f"<h3><b><span style='color:lightgreen'>💰 Available Cash (USD) </b> : </span> $ {available_cash}</h3")
printmd(f"<h3><span style='color:#FFCE1B'>🛖 Equities : </span> $ {equities}</h3")
printmd(f"<h3><b><span style='color:lightgreen'>👍🏼 Total Realised PnL</b> : </span> $ {total_pnl}</h3")
printmd(f"<h3><b><span style='color:#C04657'>🔴 IB Interest Recieved </b> : </span> $ {interest_recieved}</h3")
printmd(f"<h3><span style='color:#C04657'>🤦🏻‍♂️ Commission Paid : </span> $ {total_fees}</h3")

growth_portfolio = categorised_portfolio(portfolio, growth_symbols)
emerging_portfolio = categorised_portfolio(portfolio, emerging_symbols)
divident_portfolio = categorised_portfolio(portfolio, divident_symbols)
exited_assets = portfolio[portfolio['Quantity'] == 0.0]

# Overall Portfolio
render(portfolio[portfolio['Quantity'] != 0.0], "n03an's Portfolio: Cost Basis")

# Growth Portfolio
printmd(f"<hr /><h3><span style='color:cyan'>Growth Portfolio: 30% Average Annual Rate of Return</span></h3")
printmd(f"<p><span style='color:tomato'>Volatility : High</span></p")
printmd(f"<p><span style='color:white'>This portfolio aims to diversify into many halal and promising high-growth companies with a focus on disruptive technology.</span></p")
render(growth_portfolio, 'Growth Portfolio: Cost Basis')

# Emerging Portfolio
printmd(f"<hr /><h3><span style='color:cyan'>Emerging Portfolio: 10X in 5 Years</span></h3")
printmd(f"<p><span style='color:tomato'>Volatility : High</span></p")
render(emerging_portfolio, 'Emerging Portfolio: Cost Basis')

# Divident Portfolio
printmd(f"<hr /><h3><span style='color:cyan'>Divident Portfolio: 3% - 4% annual Dividend Yield</span></h3")
printmd(f"<p><span style='color:orange'>Volatility : Moderate</span></p")
printmd(f"<p><span style='color:white'>This portfolio aims to provide a steady stream of passive halal income from Dividends.</span></p")
render(divident_portfolio, 'Divident Portfolio: Cost Basis')

# Exited Asset
printmd(f"<hr /><h3><span style='color:cyan'>Exited Assets</span></h3")
table(exited_assets)


In [19]:
import yfinance as yf
from datetime import datetime, timedelta

data = yf.Ticker("IREN").info
print(data["fiftyTwoWeekHigh"])
print(data["fiftyTwoWeekLow"])
print(data["dayHigh"])
print(data["dayLow"])
print(data["currentPrice"])

15.92
3.56
10.49
9.805
9.94
